In [ ]:
from typing import Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


from autogen.agentchat import AssistantAgent
from autogen.tools import Tool
from autogen.interop import Interoperability

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        mcp_tools = await session.list_tools()
        interop = Interoperability()
        tools = [interop.convert_tool(tool=mcp_tool, type="mcp", session=session) for mcp_tool in mcp_tools.tools]

        agent = AssistantAgent(name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai"})
        for tool in tools:
            tool.register_for_llm(agent)

        for tool_schema in agent.llm_config["tools"]:
            print(f"Tool schema: {tool_schema}")
        agent.run(
            message="Add 123223 and 456789",
            tools=tools,
            max_turns=2,
            user_input=False,
        )